In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
# t_res='5min'; res='1km'
# res='1km'
# Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'
Np_str='50e6'


# # dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***
# res='250m'
# Np_str='150e6'

In [ ]:
#INITIALIZE DATA FUNCTION
###############################################################
def initiate_array(out_file, vars, t_chunk_size, p_chunk_size, t_size=None, p_size=None):
    if t_size is None:
        t_size = len(data['time'])  # Number of timesteps
    if p_size is None:
        p_size = len(parcel['xh'])  # Number of vertical levels

    with h5py.File(out_file, 'w') as f:
        for var_name in vars:
            if var_name not in f:
                # Set dtype conditionally
                if var_name in ['Z', 'Y', 'X']:
                    dtype = np.uint16
                elif var_name in ['A_g', 'A_c']:
                    dtype = np.bool_
                else:
                    dtype = np.float32  # or whatever your default is

                f.create_dataset(
                    var_name,
                    shape=(t_size, p_size),
                    chunks=(t_chunk_size, p_chunk_size),
                    dtype=dtype
                )

In [ ]:
def check_memory():
    import sys
    ipython_vars = ["In", "Out", "exit", "quit", "get_ipython", "ipython_vars"]
    print("Top 10 objects with highest memory usage")
    # Get a sorted list of the objects and their sizes
    mem = {
        key: round(value/1e6,2)
        for key, value in sorted(
            [
                (x, sys.getsizeof(globals().get(x)))
                for x in globals()
                if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
            ],
            key=lambda x: x[1],
            reverse=True)[:10]
    }
    print({key:f"{value} MB" for key,value in mem.items()})
    print(f"\n{round(sum(mem.values()),2)/1000} GB in use overall")

In [ ]:
#JOB ARRAY SETUP
job_array=False
job_array=True

if job_array==True:

    num_jobs=300 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(data['time']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id, num_jobs):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=1
    start_job, end_job = get_job_range(job_id, num_jobs)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')

In [ ]:
if job_array==True:
    #Indexing Array with JobArray
    data=data.isel(time=slice(start_job,end_job))
    parcel=parcel.isel(time=slice(start_job,end_job))
    #(for 150_000_000 parcels use 500-1000 jobs)

if job_array==False:
    start_job=0;end_job=len(data['time']);index_adjust=0

In [ ]:
#LOADING VARIABLES
###############################################################

In [ ]:
# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
open_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
with h5py.File(open_file, 'r') as f:
    Z = f['Z'][start_job:end_job]
    Y = f['Y'][start_job:end_job]
    X = f['X'][start_job:end_job]

# #Making Time Matrix
# rows, cols = A.shape[0], A.shape[1]
# T = np.arange(rows).reshape(-1, 1) * np.ones((1, cols), dtype=int)
check_memory()

In [ ]:
def call_variable(varname):
    if varname=='th_e':
        with h5py.File(dir + 'Variable_Calculation/' + 'theta_e'+f'_{res}_{t_res}_{Np_str}'+'.h5', 'r') as f:
            var_data = f['theta_e'][start_job:end_job]
    else:
        var_data=data[varname].data
    return var_data

In [ ]:
def make_lagrangian_array(varnames):
    # Initialize dictionaries
    var_data_dict = {varname: call_variable(varname) for varname in varnames}
    VAR = {varname: np.zeros_like(Z, dtype='float32') for varname in varnames}

    Nt = len(data['time'])
    Np = len(parcel['xh'])
    for p in np.arange(Np):
        if np.mod(p, 1e6) == 0: 
            print(f"{p}/{len(parcel['xh'])}")

        # Get Indices
        zs = Z[:, p]
        ys = Y[:, p]
        xs = X[:, p]
        ts = np.arange(Nt)

        # Loop over all variables and fill the respective VAR array
        for varname, var_data in var_data_dict.items():
            VAR[varname][:, p] = var_data[ts, zs, ys, xs]

    # Return all the arrays in a list
    return [VAR[varname] for varname in varnames]

In [ ]:
#W BUDGET VARIABLES
#########################################
varnames=['wb_hadv','wb_vadv','wb_hidiff','wb_vidiff',
          'wb_hturb','wb_vturb','wb_pgrad','wb_buoy']
[WB_HADV,WB_VADV,WB_HIDIFF,WB_VIDIFF,WB_HTURB,WB_VTURB,WB_PGRAD,WB_BUOY]=make_lagrangian_array(varnames); check_memory()

In [ ]:
# Saving Data
##############
print('Saving Data\n')
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
out_file=dir2+f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}'
if job_array==True:
    out_file+=f'_{job_id}.h5'
elif job_array==False:
    out_file+=f'.h5'

vars=['WB_HADV','WB_VADV','WB_HIDIFF','WB_VIDIFF','WB_HTURB','WB_VTURB','WB_PGRAD','WB_BUOY']
initiate_array(out_file,vars,t_chunk_size=1,p_chunk_size=100_000)

with h5py.File(out_file, 'a') as f: 
    f['WB_HADV'][:]=WB_HADV
    f['WB_VADV'][:]=WB_VADV
    f['WB_HIDIFF'][:]=WB_HIDIFF
    f['WB_VIDIFF'][:]=WB_VIDIFF
    f['WB_HTURB'][:]=WB_HTURB
    f['WB_VTURB'][:]=WB_VTURB
    f['WB_PGRAD'][:]=WB_PGRAD
    f['WB_BUOY'][:]=WB_BUOY

In [ ]:
#QV BUDGET VARIABLES
#########################################
varnames=['qvb_hadv','qvb_vadv','qvb_hidiff','qvb_vidiff',
          'qvb_hturb','qvb_vturb','qvb_mp']
[QVB_HADV,QVB_VADV,QVB_HIDIFF,QVB_VIDIFF,QVB_HTURB,QVB_VTURB,QVB_MP]=make_lagrangian_array(varnames); check_memory()

In [ ]:
# Saving Data
##############
print('Saving Data\n')
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
out_file=dir2+f'QV_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}'
if job_array==True:
    out_file+=f'_{job_id}.h5'
elif job_array==False:
    out_file+=f'.h5'

vars=['QVB_HADV','QVB_VADV','QVB_HIDIFF','QVB_VIDIFF','QVB_HTURB','QVB_VTURB','QVB_MP']
initiate_array(out_file,vars,t_chunk_size=1,p_chunk_size=100_000)

with h5py.File(out_file, 'a') as f: 
    f['QVB_HADV'][:]=QVB_HADV
    f['QVB_VADV'][:]=QVB_VADV
    f['QVB_HIDIFF'][:]=QVB_HIDIFF
    f['QVB_VIDIFF'][:]=QVB_VIDIFF
    f['QVB_HTURB'][:]=QVB_HTURB
    f['QVB_VTURB'][:]=QVB_VTURB
    f['QVB_MP'][:]=QVB_MP

In [ ]:
#TH BUDGET VARIABLES
#########################################
varnames=['ptb_hadv','ptb_vadv','ptb_hidiff','ptb_vidiff',
          'ptb_hturb','ptb_vturb','ptb_mp','ptb_rad','ptb_div','ptb_diss']
[PTB_HADV,PTB_VADV,PTB_HIDIFF,PTB_VIDIFF,PTB_HTURB,PTB_VTURB,PTB_MP,PTB_RAD,PTB_DIV,PTB_DISS]=make_lagrangian_array(varnames); check_memory()

In [ ]:
# Saving Data
##############
print('Saving Data\n')
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
out_file=dir2+f'TH_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}'
if job_array==True:
    out_file+=f'_{job_id}.h5'
elif job_array==False:
    out_file+=f'.h5'

vars=['PTB_HADV','PTB_VADV','PTB_HIDIFF','PTB_VIDIFF','PTB_HTURB','PTB_VTURB','PTB_MP',
      'PTB_RAD','PTB_DIV','PTB_DISS']
initiate_array(out_file,vars,t_chunk_size=1,p_chunk_size=100_000)

with h5py.File(out_file, 'a') as f: 
    f['PTB_HADV'][:]=PTB_HADV
    f['PTB_VADV'][:]=PTB_VADV
    f['PTB_HIDIFF'][:]=PTB_HIDIFF
    f['PTB_VIDIFF'][:]=PTB_VIDIFF
    f['PTB_HTURB'][:]=PTB_HTURB
    f['PTB_VTURB'][:]=PTB_VTURB
    f['PTB_MP'][:]=PTB_MP
    f['PTB_RAD'][:]=PTB_RAD
    f['PTB_DIV'][:]=PTB_DIV
    f['PTB_DISS'][:]=PTB_DISS

In [ ]:
#########################################
#RECOMBINE SEPERATE JOB_ARRAYS AFTER
recombine=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
# recombine=True

In [ ]:
# if recombine==True:
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
#     dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
#     out_file=dir3+f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
    
#     vars=['W_HADV','W_VADV','WB_HIDIFF','WB_VIDIFF','WB_HTURB','WB_VTURB','WB_PGRAD','WB_BUOY']
#     initiate_array(out_file,vars,t_chunk_size=100,p_chunk_size=100_000)
    
#     with h5py.File(out_file, 'r+') as f_out:
#         num_jobs=300
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,5)==0: print(f"job_id = {job_id}")
#             [a,b] = get_job_range(job_id,num_jobs)
            
#             in_file=dir2+f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}_{job_id}.h5'
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
#DASK-ENABLED
def recombine_func(in_files,out_file):
    matching_files = sorted(glob.glob(in_files))
    print('recombining')
    # %%time
    out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_0',combine='nested',phony_dims='sort')
    out.to_netcdf(out_file, engine='h5netcdf')
    
if recombine==True:
    import glob
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_files = dir2 + f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}_*.h5'
    out_file=dir3+f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5' 
    
    recombine_func(in_files,out_file)

In [ ]:
# if recombine==True:
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
#     dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
#     # out_file=dir3+f'QV_BUDGET_VARS_binary_array_{res}_{Np_str}_5min.h5'
#     out_file=dir3+f'QV_BUDGET_VARS_binary_array_{res}_{Np_str}_1min.h5'
    
#     vars=['QVB_HADV','QVB_VADV','QVB_HIDIFF','QVB_VIDIFF','QVB_HTURB','QVB_VTURB','QVB_MP']
#     initiate_array(out_file,vars,t_chunk_size=100,p_chunk_size=100_000)
    
#     with h5py.File(out_file, 'r+') as f_out:
#         num_jobs=300
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,5)==0: print(f"job_id = {job_id}")
#             [a,b] = get_job_range(job_id,num_jobs)
            
#             # in_file=dir2+f'QV_BUDGET_VARS_binary_array_{res}_{Np_str}_5min_{job_id}.h5'
#             in_file=dir2+f'QV_BUDGET_VARS_binary_array_{res}_{Np_str}_1min_{job_id}.h5'
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
#DASK-ENABLED
def recombine_func(in_files,out_file):
    matching_files = sorted(glob.glob(in_files))
    print('recombining')
    # %%time
    out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_0',combine='nested',phony_dims='sort')
    out.to_netcdf(out_file, engine='h5netcdf')
    
if recombine==True:
    import glob
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_files = dir2 + f'QV_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}_*.h5'
    out_file=dir3+f'QV_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5' 
    
    recombine_func(in_files,out_file)

In [ ]:
# if recombine==True:
#     dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
#     dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
#     # out_file=dir3+f'TH_BUDGET_VARS_binary_array_{res}_{Np_str}_5min.h5'
#     out_file=dir3+f'TH_BUDGET_VARS_binary_array_{res}_{Np_str}_1min.h5'
    
#     vars=['PTB_HADV','PTB_VADV','PTB_HIDIFF','PTB_VIDIFF','PTB_HTURB','PTB_VTURB','PTB_MP',
#           'PTB_RAD','PTB_DIV','PTB_DISS']
#     initiate_array(out_file,vars,t_chunk_size=100,p_chunk_size=100_000)
    
#     with h5py.File(out_file, 'r+') as f_out:
#         num_jobs=300
#         for job_id in np.arange(1,num_jobs+1):
#             if np.mod(job_id,5)==0: print(f"job_id = {job_id}")
#             [a,b] = get_job_range(job_id,num_jobs)
            
#             # in_file=dir2+f'TH_BUDGET_VARS_binary_array_{res}_{Np_str}_5min_{job_id}.h5'
#             in_file=dir2+f'TH_BUDGET_VARS_binary_array_{res}_{Np_str}_1min_{job_id}.h5'
#             with h5py.File(in_file, 'r') as f_in: 
#                 for var in vars:
#                     f_out[var][a:b]=f_in[var][:]

In [ ]:
#DASK-ENABLED
def recombine_func(in_files,out_file):
    matching_files = sorted(glob.glob(in_files))
    print('recombining')
    # %%time
    out=xr.open_mfdataset(matching_files,engine='h5netcdf',concat_dim='phony_dim_0',combine='nested',phony_dims='sort')
    out.to_netcdf(out_file, engine='h5netcdf')
    
if recombine==True:
    import glob
    dir2=dir+'Project_Algorithms/Lagrangian_Arrays/job_out/'
    dir3=dir+'Project_Algorithms/Lagrangian_Arrays/'
    in_files = dir2 + f'TH_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}_*.h5'
    out_file=dir3+f'TH_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5' 
    
    recombine_func(in_files,out_file)

In [ ]:
#########################################
#READING BACK IN

In [ ]:
# import h5py
# # File path to the saved data
# dir2 = dir + 'Project_Algorithms/Lagrangian_Arrays/'
# filename = f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'

# # Open the HDF5 file and read the datasets
# with h5py.File(dir2 + filename, 'r') as f:
#     WB_HADV = f['WB_HADV'][:]
#     WB_VADV = f['WB_VADV'][:]
#     WB_HIDIFF = f['WB_HIDIFF'][:]
#     WB_VIDIFF = f['WB_VIDIFF'][:]
#     WB_HTURB = f['WB_HTURB'][:]
#     WB_VTURB = f['WB_VTURB'][:]
#     WB_PGRAD = f['WB_PGRAD'][:]
#     WB_BUOY = f['WB_BUOY'][:]

# # Check memory after loading
# check_memory()


In [ ]:
# import h5py
# # File path to the saved data
# dir2 = dir + 'Project_Algorithms/Lagrangian_Arrays/'
# filename = f'QV_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'

# # Open the HDF5 file and read the datasets
# with h5py.File(dir2 + filename, 'r') as f:
#     QVB_HADV = f['QVB_HADV'][:]
#     QVB_VADV = f['QVB_VADV'][:]
#     QVB_HIDIFF = f['QVB_HIDIFF'][:]
#     QVB_VIDIFF = f['QVB_VIDIFF'][:]
#     QVB_HTURB = f['QVB_HTURB'][:]
#     QVB_VTURB = f['QVB_VTURB'][:]
#     QVB_MP = f['QVB_MP'][:]

# # Check memory after loading
# check_memory()


In [ ]:
# import h5py
# # File path to the saved data
# dir2 = dir + 'Project_Algorithms/Lagrangian_Arrays/'
# filename = f'TH_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'

# # Open the HDF5 file and read the datasets
# with h5py.File(dir2 + filename, 'r') as f:
#     PTB_HADV = f['PTB_HADV'][:]
#     PTB_VADV = f['PTB_VADV'][:]
#     PTB_HIDIFF = f['PTB_HIDIFF'][:]
#     PTB_VIDIFF = f['PTB_VIDIFF'][:]
#     PTB_HTURB = f['PTB_HTURB'][:]
#     PTB_VTURB = f['PTB_VTURB'][:]
#     PTB_MP = f['PTB_MP'][:]
#     PTB_RAD = f['PTB_RAD'][:]
#     PTB_DIV = f['PTB_DIV'][:]
#     PTB_DISS = f['PTB_DISS'][:]

# # Check memory after loading
# check_memory()

In [ ]:
#########################################
#TESTING

In [ ]:
# t=100
# dir2 = dir + 'Project_Algorithms/Lagrangian_Arrays/'
# filename = f'W_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(dir2 + filename, 'r') as f:
#     WB_HADV = f['WB_HADV'][t]
#     WB_VADV = f['WB_VADV'][t]
#     WB_HIDIFF = f['WB_HIDIFF'][t]
#     WB_VIDIFF = f['WB_VIDIFF'][t]
#     WB_HTURB = f['WB_HTURB'][t]
#     WB_VTURB = f['WB_VTURB'][t]
#     WB_PGRAD = f['WB_PGRAD'][t]
#     WB_BUOY = f['WB_BUOY'][t]


In [ ]:
# dir2 = dir + 'Project_Algorithms/Lagrangian_Arrays/'
# filename = f'QV_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(dir2 + filename, 'r') as f:
#     QVB_HADV = f['QVB_HADV'][t]
#     QVB_VADV = f['QVB_VADV'][t]
#     QVB_HIDIFF = f['QVB_HIDIFF'][t]
#     QVB_VIDIFF = f['QVB_VIDIFF'][t]
#     QVB_HTURB = f['QVB_HTURB'][t]
#     QVB_VTURB = f['QVB_VTURB'][t]
#     QVB_MP = f['QVB_MP'][t]

In [ ]:
# import h5py
# # File path to the saved data
# dir2 = dir + 'Project_Algorithms/Lagrangian_Arrays/'
# filename = f'TH_BUDGET_VARS_binary_array_{res}_{t_res}_{Np_str}.h5'

# # Open the HDF5 file and read the datasets
# with h5py.File(dir2 + filename, 'r') as f:
#     PTB_HADV = f['PTB_HADV'][t]
#     PTB_VADV = f['PTB_VADV'][t]
#     PTB_HIDIFF = f['PTB_HIDIFF'][t]
#     PTB_VIDIFF = f['PTB_VIDIFF'][t]
#     PTB_HTURB = f['PTB_HTURB'][t]
#     PTB_VTURB = f['PTB_VTURB'][t]
#     PTB_MP = f['PTB_MP'][t]
#     PTB_RAD = f['PTB_RAD'][t]
#     PTB_DIV = f['PTB_DIV'][t]
#     PTB_DISS = f['PTB_DISS'][t]

# # Check memory after loading
# check_memory()

In [ ]:
# dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
# with h5py.File(dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5', 'r') as f:
#     # Load the dataset by its name
#     Z = f['Z'][t]
#     Y = f['Y'][t]
#     X = f['X'][t]

In [ ]:
# t,p=100,1000
# def test(t,p,VAR,var):
#     z=Z[p];y=Y[p];x=X[p]
#     out=data[var].isel(time=t,zh=z,yh=y,xh=x).data
#     print(VAR[p],out)
# def test2(t,p,VAR,var):
#     z=Z[p];y=Y[p];x=X[p]
#     out=data[var].isel(time=t,zf=z,yh=y,xh=x).data
#     print(VAR[p],out)

# test2(t,p,WB_HADV,'wb_hadv')
# test(t,p,QVB_HADV,'qvb_hadv')
# test(t,p,PTB_HADV,'ptb_hadv')
